In [1]:
import json
from PIL import Image
import pymupdf
import sys
import os
import re
import fitz
from collections import defaultdict

from tqdm import tqdm
from fuzzysearch import find_near_matches

In [2]:


ROOT = "../../"

BOOK_PATH =  ROOT + "files/data-science_book/data-science-content.pdf"
TEX_PATH = ROOT + "files/data-science_book/outputs/data-science_cleaned_2p_bib_first_index_after_parallel.tex"
INDEX_PATH = ROOT + "files/data-science_book/data-science-index.pdf"
OUTPUT_TEX_PATH = ROOT + "files/data-science_book/outputs/data-science_cleaned_2p_bib_first_index_after_parallel_indexed.tex"



 

In [ ]:
# for table of contents
TOC_PATH  = ROOT + "files/data-science_book/data-science_toc.pdf"
all_toc_text = ""
for page_num in range(len(toc)):
    page = toc[page_num]
    text = page.get_text()
    all_toc_text += f"\n--- Page {page_num + 1} ---\n"
    all_toc_text += text.strip() + "\n"

toc.close()

In [3]:
toc = pymupdf.open("../../files/data-science_book/data-science-content.pdf")

table = toc.get_toc()

In [4]:
table

[[1, '1\rWhat is Data Science?', 1],
 [2, '1.1 Computer Science, Data Science, and Real', 2],
 [2, 'Science', 2],
 [2, '1.2 Asking Interesting Questions from Data', 4],
 [3, '1.2.1 The Baseball Encyclopedia', 5],
 [3, '1.2.2 The Internet Movie Database (IMDb)', 7],
 [3, '1.2.3 Google Ngrams', 10],
 [3, '1.2.4 New York Taxi Records', 11],
 [2, '1.3 Properties of Data', 14],
 [3, '1.3.1 Structured vs. Unstructured Data', 14],
 [3, '1.3.2 Quantitative vs. Categorical Data', 15],
 [3, '1.3.3 Big Data vs. Little Data', 15],
 [2, '1.4 Classi\x0ccation and Regression', 16],
 [2, '1.5 Data Science Television: The Quant Shop', 17],
 [3, '1.5.1 Kaggle Challenges', 19],
 [2, '1.6 About the War Stories', 19],
 [2, '1.7 War Story: Answering the Right Question', 21],
 [2, '1.8 Chapter Notes', 22],
 [2, '1.9 Exercises', 23],
 [1, '2\rMathematical Preliminaries', 26],
 [2, '2.1 Probability', 26],
 [3, '2.1.1 Probability vs. Statistics', 28],
 [3, '2.1.2 Compound Events and Independence', 29],
 [3, '2.

In [5]:
def clean_and_merge_toc(toc):
    cleaned_toc = []

    def remove_numbering(title):
        title = title.replace('\r', '').replace('\x0c', 'fi')
        return re.sub(r'^\s*\d+(\.\d+)*\s*', '', title).strip()

    for entry in toc:
        level, title, page = entry
        title_clean = title.replace('\r', '').replace('\x0c', 'fi').strip()

        if cleaned_toc:
            prev_level, prev_title, prev_page = cleaned_toc[-1]
            # If same level and same page, and this one doesn’t start with a number — join
            if level == prev_level and page == prev_page and not re.match(r'^\s*\d+(\.\d+)*', title_clean):
                cleaned_toc[-1] = (prev_level, prev_title + " " + title_clean, prev_page)
                continue

        cleaned_title = remove_numbering(title_clean)
        cleaned_toc.append((level, cleaned_title, page))

    return cleaned_toc

In [100]:
toc = clean_and_merge_toc(table)

Original title: 1What is Data Science?
After replacing: 1What is Data Science?
Original title: 1.1 Computer Science, Data Science, and Real
After replacing: 1.1 Computer Science, Data Science, and Real
Original title: 1.2 Asking Interesting Questions from Data
After replacing: 1.2 Asking Interesting Questions from Data
Original title: 1.2.1 The Baseball Encyclopedia
After replacing: 1.2.1 The Baseball Encyclopedia
Original title: 1.2.2 The Internet Movie Database (IMDb)
After replacing: 1.2.2 The Internet Movie Database (IMDb)
Original title: 1.2.3 Google Ngrams
After replacing: 1.2.3 Google Ngrams
Original title: 1.2.4 New York Taxi Records
After replacing: 1.2.4 New York Taxi Records
Original title: 1.3 Properties of Data
After replacing: 1.3 Properties of Data
Original title: 1.3.1 Structured vs. Unstructured Data
After replacing: 1.3.1 Structured vs. Unstructured Data
Original title: 1.3.2 Quantitative vs. Categorical Data
After replacing: 1.3.2 Quantitative vs. Categorical Data
Or

In [88]:
toc

[(1, 'What is Data Science?', 1),
 (2, 'Computer Science, Data Science, and Real Science', 2),
 (2, 'Asking Interesting Questions from Data', 4),
 (3, 'The Baseball Encyclopedia', 5),
 (3, 'The Internet Movie Database (IMDb)', 7),
 (3, 'Google Ngrams', 10),
 (3, 'New York Taxi Records', 11),
 (2, 'Properties of Data', 14),
 (3, 'Structured vs. Unstructured Data', 14),
 (3, 'Quantitative vs. Categorical Data', 15),
 (3, 'Big Data vs. Little Data', 15),
 (2, 'Classication and Regression', 16),
 (2, 'Data Science Television: The Quant Shop', 17),
 (3, 'Kaggle Challenges', 19),
 (2, 'About the War Stories', 19),
 (2, 'War Story: Answering the Right Question', 21),
 (2, 'Chapter Notes', 22),
 (2, 'Exercises', 23),
 (1, 'Mathematical Preliminaries', 26),
 (2, 'Probability', 26),
 (3, 'Probability vs. Statistics', 28),
 (3, 'Compound Events and Independence', 29),
 (3, 'Conditional Probability', 30),
 (3, 'Probability Distributions', 31),
 (2, 'Descriptive Statistics', 33),
 (3, 'Centrality M

In [60]:
toc_dict = {title.lower(): level for level, title, _ in toc}


In [62]:
toc_dict['classification and regression']

KeyError: 'classification and regression'

In [85]:
title = "Classi\x0ccation and Regression"
title_clean = title.replace('\r', '').replace('\x0c', 'fi').strip()
title_clean

'Classification and Regression'

In [50]:
all_toc_text

'\n--- Page 1 ---\nContents\n1\nWhat is Data Science?\n1\n1.1\nComputer Science, Data Science, and Real Science . . . . . . . .\n2\n1.2\nAsking Interesting Questions from Data . . . . . . . . . . . . . .\n4\n1.2.1\nThe Baseball Encyclopedia\n. . . . . . . . . . . . . . . . .\n5\n1.2.2\nThe Internet Movie Database (IMDb) . . . . . . . . . . .\n7\n1.2.3\nGoogle Ngrams . . . . . . . . . . . . . . . . . . . . . . . .\n10\n1.2.4\nNew York Taxi Records . . . . . . . . . . . . . . . . . . .\n11\n1.3\nProperties of Data . . . . . . . . . . . . . . . . . . . . . . . . . .\n14\n1.3.1\nStructured vs. Unstructured Data\n. . . . . . . . . . . . .\n14\n1.3.2\nQuantitative vs. Categorical Data\n. . . . . . . . . . . . .\n15\n1.3.3\nBig Data vs. Little Data . . . . . . . . . . . . . . . . . . .\n15\n1.4\nClassiﬁcation and Regression\n. . . . . . . . . . . . . . . . . . . .\n16\n1.5\nData Science Television: The Quant Shop . . . . . . . . . . . . .\n17\n1.5.1\nKaggle Challenges\n. . . . . . . . . . . .

In [48]:
all_text

'\n--- Page 1 ---\nContents\n1\nWhat is Data Science?\n1\n1.1\nComputer Science, Data Science, and Real Science . . . . . . . .\n2\n1.2\nAsking Interesting Questions from Data . . . . . . . . . . . . . .\n4\n1.2.1\nThe Baseball Encyclopedia\n. . . . . . . . . . . . . . . . .\n5\n1.2.2\nThe Internet Movie Database (IMDb) . . . . . . . . . . .\n7\n1.2.3\nGoogle Ngrams . . . . . . . . . . . . . . . . . . . . . . . .\n10\n1.2.4\nNew York Taxi Records . . . . . . . . . . . . . . . . . . .\n11\n1.3\nProperties of Data . . . . . . . . . . . . . . . . . . . . . . . . . .\n14\n1.3.1\nStructured vs. Unstructured Data\n. . . . . . . . . . . . .\n14\n1.3.2\nQuantitative vs. Categorical Data\n. . . . . . . . . . . . .\n15\n1.3.3\nBig Data vs. Little Data . . . . . . . . . . . . . . . . . . .\n15\n1.4\nClassiﬁcation and Regression\n. . . . . . . . . . . . . . . . . . . .\n16\n1.5\nData Science Television: The Quant Shop . . . . . . . . . . . . .\n17\n1.5.1\nKaggle Challenges\n. . . . . . . . . . . .

##################

In [ ]:
og_book_pdf = pymupdf.open(BOOK_PATH)

In [4]:
len(og_book_pdf)

438

In [6]:
book_pdf = pymupdf.open(INDEX_PATH)

In [9]:
book_pdf[0]

page 0 of ../../files/data-science_book/data-science-index.pdf

In [ ]:
# handling subindex entries

def extract_index_entries(pdf_path):
    doc = fitz.open(pdf_path)
    index = defaultdict(list)
    current_main_term = None
    counter = 1
    for page in doc:
        text_dict = page.get_text("dict")
        # print("Text Dict:", text_dict)

        for block in text_dict["blocks"]:
            # print("Block:", block)

            for line in block["lines"]:
                # print("Line:", line)
                for span in line["spans"]:
                    text = span["text"].strip()
                    if text:
                        print("Span: ", span) # this has index entry
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        break
    
        continue
        blocks.sort(key=lambda b: (b[0], b[1]))  # sort top-down, then left-right
        print("Page number:", counter)
        counter += 1
        for block in blocks:
            text = block[4].strip()
            if not text:
                continue
            print(block)
            # Attempt to extract page numbers from end of line
            
            
            for match in pattern.finditer(text):
                term = match.group(1).strip()
                pages = [int(p) for p in re.findall(r"\d+", match.group(2))]
            
                is_subentry = block[0] > 50  # x-coordinate threshold (tweak as needed)
                if is_subentry and current_main_term:
                    subentry = f"{current_main_term}!{term}"
                    index[subentry].extend(pages)
                else:
                    current_main_term = term
                    index[term].extend(pages)
                   
            
            # match = re.match(pattern, text)
            # if match:
            #     entry, pages = match.groups()
            #     page_numbers = [int(p) for p in re.findall(r"\d+", pages)]

            #     is_subentry = block[0] > 50  # x-coordinate threshold (tweak as needed)

            #     if is_subentry and current_main_term:
            #         subentry = f"{current_main_term}!{entry}"
            #         index[subentry].extend(page_numbers)
            #     else:
            #         current_main_term = entry
            #         index[entry].extend(page_numbers)


    return dict(index)

In [23]:
new_index_entries = extract_index_entries(INDEX_PATH)

Span:  {'size': 24.787099838256836, 'flags': 20, 'bidi': 0, 'char_flags': 16, 'font': 'RfrbdtDgcwxnCMBX12', 'color': 0, 'alpha': 255, 'ascender': 0.6940000057220459, 'descender': -0.1940000057220459, 'text': 'Index', 'origin': (79.93990325927734, 106.10400390625), 'bbox': (79.93990325927734, 86.73210144042969, 148.72410583496094, 111.51920318603516)}
Span:  {'size': 9.962599754333496, 'flags': 4, 'bidi': 0, 'char_flags': 16, 'font': 'XgmrvdCfsxnjCMR10', 'color': 0, 'alpha': 255, 'ascender': 0.6940000057220459, 'descender': -0.1940000057220459, 'text': 'A/B testing, 86', 'origin': (79.93990325927734, 169.864990234375), 'bbox': (79.93990325927734, 162.0789031982422, 148.16378784179688, 172.04150390625)}
Span:  {'size': 9.962599754333496, 'flags': 4, 'bidi': 0, 'char_flags': 16, 'font': 'XgmrvdCfsxnjCMR10', 'color': 0, 'alpha': 255, 'ascender': 0.6940000057220459, 'descender': -0.1940000057220459, 'text': 'Aaron Schwartz case, 68', 'origin': (79.93990325927734, 181.82000732421875), 'bbox'

In [33]:
new_index_entries

{'A/B testing': [86],
 'A/B testing!Aaron Schwartz case': [68],
 'A/B testing!AB testing': [137],
 'A/B testing!academic data': [66],
 'A/B testing!accuracy': [215, 228],
 'A/B testing!activation function': [380],
 'A/B testing!AdaBoost': [364],
 'A/B testing!add-one discounting': [357],
 'A/B testing!agglomerative cluster trees': [338],
 'A/B testing!aggregation mechanisms': [83],
 'A/B testing!Akaike information criterion': [289],
 'A/B testing!average link': [340],
 'A/B testing!Babbage, Charles': [57, 90],
 'A/B testing!backpropagation': [382],
 'A/B testing!Bacon, Kevin': [9],
 'A/B testing!bag of words': [14],
 'A/B testing!bagging': [362],
 'A/B testing!balanced training classes': [295],
 'A/B testing!bar charts': [179, 179],
 'A/B testing!best practices': [181, 186, 175, 181, 177],
 'A/B testing!stacked': [181],
 'A/B testing!Barzun, Jacques': [5],
 'A/B testing!baseball encyclopedia': [5],
 'A/B testing!baseline models': [210],
 'A/B testing!algorithm analysis': [397],
 'A/B t

In [29]:
len(new_index_entries)

795

In [21]:
index_data = {}
for i in range(len(book_pdf)):
    page = book_pdf[i]
    index_data[i] = page.get_text("text")

In [22]:
index_data[0]

'Index\nA/B testing, 86\nAaron Schwartz case, 68\nAB testing, 137\nacademic data, 66\naccuracy, 215, 228\nactivation function, 380\nAdaBoost, 364\nadd-one discounting, 357\nagglomerative cluster trees, 338\naggregation mechanisms, 83\nAkaike information criterion, 289,\n335\nalgorithm analysis, 397\nAmazon Turk, 67, 84\ntasks assigned, 85\nTurkers, 84\nAmerican basketball players, 97\nanalogies, 312\nanchoring, 82\nangular distance, 310\nAnscombe’s Quartet, 159\nAOL, 64\nAPI, 65\nApple iPhone sales, 34\napplication program interfaces, 65\narea under the ROC curve, 219\nAristotle, 326, 327\nArrow’s impossibility theorem, 84,\n114\nartifacts, 69\nAscombe quartet, 272\nasking interesting questions, 4\nassociativity, 244\nautocorrelation, 46\naverage link, 340\nBabbage, Charles, 57, 90\nbackpropagation, 382\nBacon, Kevin, 9\nbag of words, 14\nbagging, 362\nbalanced training classes, 295\nbar charts, 179\nbest practices, 181\nstacked, 181\nBarzun, Jacques, 5\nbaseball encyclopedia, 5\nbasel

In [23]:
pattern = re.compile(r"(.+?),\s*((?:\d+,?\s*)+)")

In [24]:
index = {}

for page, text in index_data.items():
    for match in pattern.finditer(text):
        term = match.group(1).strip()
        pages = [int(p) for p in re.findall(r"\d+", match.group(2))]
        index[term] = pages

In [25]:
len(index)

792

In [12]:
index["word embeddings"]

[254, 383]

In [14]:
with open(TEX_PATH, "r") as f:
    latex_content = f.read()

In [15]:
page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', latex_content)
page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', latex_content))}


In [16]:
page_breaks

['3',
 '5',
 '7',
 '10',
 '12',
 '14',
 '16',
 '19',
 '21',
 '23',
 '25',
 '30',
 '34',
 '36',
 '38',
 '41',
 '46',
 '48',
 '52',
 '56',
 '59',
 '61',
 '63',
 '65',
 '67',
 '67',
 '68',
 '69',
 '71',
 '73',
 '73',
 '74',
 '75',
 '75',
 '76',
 '77',
 '79',
 '81',
 '83',
 '85',
 '87',
 '89',
 '92',
 '97',
 '100',
 '102',
 '104',
 '107',
 '109',
 '111',
 '114',
 '116',
 '118',
 '122',
 '126',
 '133',
 '135',
 '135',
 '136',
 '137',
 '139',
 '142',
 '144',
 '147',
 '152',
 '156',
 '159',
 '159',
 '160',
 '161',
 '163',
 '168',
 '171',
 '171',
 '172',
 '174',
 '178',
 '180',
 '186',
 '188',
 '190',
 '192',
 '197',
 '202',
 '204',
 '206',
 '208',
 '210',
 '212',
 '212',
 '213',
 '214',
 '216',
 '218',
 '218',
 '220',
 '222',
 '224',
 '226',
 '228',
 '230',
 '232',
 '236',
 '239',
 '241',
 '241',
 '242',
 '243',
 '245',
 '247',
 '249',
 '251',
 '254',
 '260',
 '260',
 '261',
 '262',
 '265',
 '269',
 '273',
 '273',
 '274',
 '276',
 '278',
 '281',
 '283',
 '285',
 '287',
 '291',
 '293',
 '296',

In [ ]:
page_positions

In [71]:
# correct the naming issue in index
# Create a new dictionary to store updated terms
updated_index = {}

for term, pages in index.items():
    if "," in term:  
        parts = term.split(", ")
        if len(parts) == 2:  # Ensure it's a "Last, First" format
            corrected_name = f"{parts[1]} {parts[0]}"  # Convert to "First Last"
            updated_index[corrected_name] = pages  # Store corrected term
        else:
            updated_index[term] = pages  # Keep unchanged terms
    else:
        updated_index[term] = pages  # Keep unchanged terms

# Now, index is updated with corrected names
index = updated_index

index

{'A/B testing': [86],
 'Aaron Schwartz case': [68],
 'AB testing': [137],
 'academic data': [66],
 'accuracy': [215, 228],
 'activation function': [380],
 'AdaBoost': [364],
 'add-one discounting': [357],
 'agglomerative cluster trees': [338],
 'aggregation mechanisms': [83],
 'Akaike information criterion': [289, 335],
 'algorithm analysis': [397],
 'Amazon Turk': [67, 84],
 'tasks assigned': [85],
 'Turkers': [84],
 'American basketball players': [97],
 'analogies': [312],
 'anchoring': [82],
 'angular distance': [310],
 'Anscombe’s Quartet': [159],
 'AOL': [64],
 'API': [65],
 'Apple iPhone sales': [34],
 'application program interfaces': [65],
 'area under the ROC curve': [219],
 'Aristotle': [326, 327],
 'Arrow’s impossibility theorem': [84, 114],
 'artifacts': [69],
 'Ascombe quartet': [272],
 'asking interesting questions': [4],
 'associativity': [244],
 'autocorrelation': [46],
 'average link': [340],
 'Charles Babbage': [57, 90],
 'backpropagation': [382],
 'Kevin Bacon': [9],

In [17]:
def find_closest_page(page,page_breaks ,page_positions, is_forward=True):

    if str(page) in page_breaks:
        return page_positions[page]
    if is_forward:
        bound = len(og_book_pdf)
        forward = page
        while (forward) <= bound:
            if str(forward) in page_breaks:
                return page_positions[forward]
            forward += 1
        return -1
    
    else:
        bound = 0
        backward = page
        while (backward) >= bound:
            if str(backward) in page_breaks:
                return page_positions[backward]
            backward -= 1
        return 0  # No valid page found


In [ ]:
find_closest_page(359)

In [ ]:
def add_indexes(latex_content, index): 
    matched = 0
    not_matched = 0
    not_found_terms = {}  # Dictionary to store terms not found along with page numbers

    for index_term, pages in tqdm(index.items()):
        # check = False
        # if index_term == 'application':
        #     check = True
        for page in pages:
            # upadte page number indexes as terms will be added

            page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', latex_content)
            page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', latex_content))}

            upper_bound = find_closest_page(page+1, page_breaks, page_positions ,True)
            lower_bound = find_closest_page(page-2, page_breaks, page_positions ,False)

            page_content = latex_content[lower_bound:upper_bound]
            
            # match = re.search(r'\b' + re.escape(index_term) + r'\b', page_content, re.IGNORECASE)
            match = re.search(  re.escape(index_term), page_content, re.IGNORECASE)

            if match:
                term_start = lower_bound + match.start()  # Adjust index relative to book_text
                term_end = term_start + len(index_term)
                indexed_term = "\index{" + index_term + "}"
                latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                matched += 1
            else:
                # try fuzzy matching
                # term_length = len(index_term)
                # max_dist = max(1, term_length // 3)  # 1/3rd of term length
                # near_matches = find_near_matches(index_term, page_content, max_l_dist=max_dist)

                # if near_matches:
                #     best_match = min(near_matches, key=lambda x: x.dist)  # Get the closest match
                #     term_start = lower_bound + best_match.start  # Adjust index relative to book_text
                #     term_end = term_start + term_length
                #     indexed_term = "\index{" + index_term + "}"
                #     latex_content = latex_content[:term_start] + index_term + indexed_term + latex_content[term_end:]
                #     matched += 1
                #     continue

                # index_position = lower_bound + (upper_bound - lower_bound) // 2
                # indexed_term = "\index{" + index_term + "}"
                # latex_content = latex_content[:index_position] + indexed_term + latex_content[index_position:]
                
                not_matched += 1
                if index_term not in not_found_terms:
                    not_found_terms[index_term] = []
                not_found_terms[index_term].append(page)

    print(f"Matched: {matched}, Not Matched: {not_matched}")
    return latex_content, not_found_terms

In [ ]:
# indexed_latex_content = add_indexes(latex_content, index) #old

#100%|██████████| 792/792 [00:00<00:00, 4871.83it/s]
# Matched: 244, Not Matched: 703


100%|██████████| 792/792 [00:00<00:00, 4871.83it/s]

Matched: 244, Not Matched: 703


In [ ]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v2 with \b

100%|██████████| 792/792 [00:00<00:00, 2803.82it/s]

Matched: 494, Not Matched: 453


In [28]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v3 without \b

100%|██████████| 792/792 [00:00<00:00, 3388.47it/s]

Matched: 505, Not Matched: 442


In [32]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v4 added fuzzy matching

100%|██████████| 792/792 [00:11<00:00, 68.91it/s] 

Matched: 621, Not Matched: 326


In [ ]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v5 added fuzzy matching with larger range

100%|██████████| 792/792 [00:16<00:00, 49.11it/s] 

Matched: 843, Not Matched: 104


In [58]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v6 updated naming indexes

100%|██████████| 792/792 [00:17<00:00, 45.55it/s] 

Matched: 871, Not Matched: 76


In [21]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v7 updating page_positions at every iteration

100%|██████████| 792/792 [00:00<00:00, 882.47it/s]

Matched: 777, Not Matched: 170


In [ ]:
A/B Testing

$ A / B $ Testing

In [67]:
not_found_terms

{'Body Mass Index': [177],
 'types': [14],
 'decision tree classiﬁers': [299],
 'Richard W. Hamming': [1],
 'overlap percentage': [137],
 'bad examples': [183],
 'program ﬂow graph': [322],
 'university rankings': [100],
 'records from New York': [11],
 'critiquing': [189]}

In [23]:
filtered_not_found_terms = {term: pages for term, pages in not_found_terms.items() if "," in term}


In [27]:
corrected_names = {}

for term, pages in filtered_not_found_terms.items():
    if "," in term:  
        parts = term.split(", ")
        if len(parts) == 2:  # Ensure it's a "Last, First" format
            corrected_name = f"{parts[1]} {parts[0]}"  # Convert to "First Last"
            corrected_names[corrected_name] = pages  # Keep the pages same

print(corrected_names)

{'Charles Babbage': [57, 90], 'Kevin Bacon': [9], 'Josh Blumenstock': [27], 'George Box': [201], 'Lewis Carroll': [423], 'Bill Clinton': [326], 'Bill] Clinton': [326], 'Chevalier de M´er´e': [29], 'Friedrich Engels': [391], 'Pierre de Fermat': [30], 'Richard Feynman': [229], 'Bill Gates': [130], 'Charles Goodhart': [303], 'Dorian Gray': [251], 'Richard W. Hamming': [1], 'Adolf Hitler': [326], 'Michael [136] Jackson': [262], 'Andrew Lang': [267], 'Abraham Lincoln': [242], 'Carl Linnaeus': [326], 'Frederick Mosteller': [121], 'Richard Nixon': [326], 'Barack Obama': [326], 'Barack [91] Obama': [327], 'Barack] Obama': [326], 'William of Occam': [202], 'Blaise Pascal': [30], 'Ronald Reagan': [326], 'Franklin D. Roosevelt': [326], 'Jan Schaumann': [351], 'William Shakespeare': [326], 'Britney [566] Spears': [262], 'James Surowiecki': [82], 'Edward Tufte': [162]}


In [ ]:
def add_indexes_for_names(latex_content, index): 
    # 2nd version to get detailed information regarding the terms which were not found in the first check
    matched = 0      
    not_matched = 0  
    not_found_terms = {}  # Dictionary to store terms not found along with page numbers

    for index_term, pages in (index.items()):
        # check = False
        # if index_term == 'application':
        #     check = True
        print("Checking for word :" + index_term)
        for page in pages:
            print(" Current page : " , page)
            upper_bound = find_closest_page(page, True)
            lower_bound = find_closest_page(page-1, False)

            page_content = latex_content[lower_bound:upper_bound]
            
            # match = re.search(r'\b' + re.escape(index_term) + r'\b', page_content, re.IGNORECASE)
            match = re.search(  re.escape(index_term), page_content, re.IGNORECASE)

            # if check:
            #     print("Match found : " + match.group(0))
            #     print("Lowerbound : ", lower_bound)
            #     print("Upperbound : ", upper_bound)
            #     print("Page : ", page)
            if match:
                # print("Normal Match found")
                term_start = lower_bound + match.start()  # Adjust index relative to book_text
                term_end = term_start + len(index_term)
                indexed_term = "\index{" + index_term + "}"
                latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                matched += 1
            else:
                # print("Fuzzy Match Check")
                # try fuzzy matching
                term_length = len(index_term)
                max_dist = max(1, term_length // 3)  # 1/3rd of term length
                near_matches = find_near_matches(index_term, page_content, max_l_dist=max_dist)

                if near_matches:
                    # print("Fuzzy Match found")
                    best_match = min(near_matches, key=lambda x: x.dist)  # Get the closest match
                    term_start = lower_bound + best_match.start  # Adjust index relative to book_text
                    term_end = term_start + term_length
                    indexed_term = "\index{" + index_term + "}"
                    latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                    matched += 1
                    continue
                
                print("No Match found")
                print("Page Content:")
                print(page_content)

                not_matched += 1
                if index_term not in not_found_terms:
                    not_found_terms[index_term] = []
                not_found_terms[index_term].append(page)


    print(f"Matched: {matched}, Not Matched: {not_matched}")
    return latex_content, not_found_terms

In [35]:
indexed_latex_content_2, not_found_terms_2= add_indexes_for_names(indexed_latex_content, not_found_terms_2) #v4 added fuzzy matching with larger range

Checking for word :Lewis Carroll
 Current page :  423
No Match found
Page Content:
in aggregated data: It is not enough to delete names, addresses, and identity numbers to maintain privacy in a data set. Even anonymized data can be effectively de-anonymized in clever ways, by using orthogonal data sources. Consider the taxi data set we introduced in Section 1.6. It never contained any passenger identifier information in the first place. Yet it does provide pickup GPS coordinates to a resolution which might pinpoint a particular house as the source, and a particular strip joint as the destination. Now we have a pretty good idea who made that trip, and an equally good idea who might be interested in this information if the bloke were married.\\
A related experiment identified particular taxi trips taken by celebrities, so as to figure out their destination and how well they tipped Gay14. By using Google to find paparazzi photographs of celebrities getting into taxis and extracting the ti

In [22]:
os.makedirs(os.path.dirname(OUTPUT_TEX_PATH), exist_ok=True)
with open(OUTPUT_TEX_PATH, "w", encoding="utf-8") as file:
     file.write(indexed_latex_content)

In [38]:
index['application']

[359]

In [76]:
ROOT = "../../"

BOOK_PATH =  ROOT + "files/algorithms_book/algorithms-content.pdf"
TEX_PATH = ROOT + "files/algorithms_book/outputs/algorithms-content.tex"
INDEX_PATH = ROOT + "files/algorithms_book/algorithms-index.pdf"
OUTPUT_TEX_PATH = ROOT + "files/dsf_book/outputs/aalgorithms-indexed-content.tex"

In [77]:
og_book_pdf = pymupdf.open(BOOK_PATH)

In [78]:
len(og_book_pdf)

782

In [79]:
book_pdf = pymupdf.open(INDEX_PATH)

In [80]:
index_data = {}
for i in range(len(book_pdf)):
    page = book_pdf[i]
    index_data[i] = page.get_text("text")

In [81]:
index_data[0]

'Index\nϵ-moves, 703\n#P-completeness, 476, 548\n0/1 knapsack problem, 497\n2/3 tree, 443\n3-SAT, 368\nabove-below test, 475, 624\nabracadabra, 686\nabstract graph type, 454\nacademic institutions – licensing,\n713\nacceptance-rejection method, 488\nAckerman function, 459\nacyclic graph, 199\nacyclic subgraph, 618\nAda, 439\nadaptive compression algorithms,\n695\nAdaptive Simulated Annealing\n(ASA), 481\naddition, 493\nadjacency list, 204, 452\nadjacency matrix, 203, 452\nadjacent swaps, 520\nAdvanced Encryption Standard,\n697\nadvice – caveat, 438\naesthetically pleasing drawings, 574\naggregate range queries, 642\nagrep, 691\nAho–Corasick algorithm, 686\nair travel pricing, 125\nairline distance metric, 393\nairline scheduling, 482, 682\nalgorist, 22\nAlgorist Technologies—consulting,\n718\nalgorithm design, 429\nalgorithmic resources, 713\naligning DNA sequences, 706\nalignment costs, 689\nall-pairs shortest path, 261, 452,\n556\nalpha-beta pruning, 160, 510\nalpha-shapes, 628\namor

In [82]:
pattern = re.compile(r"(.+?),\s*((?:\d+,?\s*)+)")

In [83]:
index = {}

for page, text in index_data.items():
    lines = [line.strip() for line in text.split("\n") if line.strip()]

    for line in lines:
        match = pattern.match(line)
        if match:
            term = match.group(1).strip()
            pages = [int(p) for p in re.findall(r"\d+", match.group(2))]
            index[term] = pages

    # for match in pattern.finditer(text):
    #     term = match.group(1).strip()
    #     pages = [int(p) for p in re.findall(r"\d+", match.group(2))]
    #     index[term] = pages

In [84]:
index

{'ϵ-moves': [703],
 '#P-completeness': [476, 548],
 '0/1 knapsack problem': [497],
 '2/3 tree': [443],
 '3-SAT': [368],
 'above-below test': [475, 624],
 'abracadabra': [686],
 'abstract graph type': [454],
 'acceptance-rejection method': [488],
 'Ackerman function': [459],
 'acyclic graph': [199],
 'acyclic subgraph': [618],
 'Ada': [439],
 '(ASA)': [481],
 'addition': [493],
 'adjacency list': [204, 452],
 'adjacency matrix': [203, 452],
 'adjacent swaps': [520],
 'advice – caveat': [438],
 'aesthetically pleasing drawings': [574],
 'aggregate range queries': [642],
 'agrep': [691],
 'Aho–Corasick algorithm': [686],
 'air travel pricing': [125],
 'airline distance metric': [393],
 'airline scheduling': [482, 682],
 'algorist': [22],
 'algorithm design': [429],
 'algorithmic resources': [713],
 'aligning DNA sequences': [706],
 'alignment costs': [689],
 'all-pairs shortest path': [261, 452],
 'alpha-beta pruning': [160, 510],
 'alpha-shapes': [628],
 'amortized analysis': [444],
 'an

In [85]:
with open(TEX_PATH, "r") as f:
    latex_content = f.read()

In [86]:
page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', latex_content)
page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', latex_content))}


In [ ]:
# updated_index = {}

# for term, pages in index.items():
#     if "," in term:  
#         parts = term.split(", ")
#         if len(parts) == 2:  # Ensure it's a "Last, First" format
#             corrected_name = f"{parts[1]} {parts[0]}"  # Convert to "First Last"
#             updated_index[corrected_name] = pages  # Store corrected term
#         else:
#             updated_index[term] = pages  # Keep unchanged terms
#     else:
#         updated_index[term] = pages  # Keep unchanged terms

# # Now, index is updated with corrected names
# index = updated_index

In [87]:
def find_closest_page(page,page_breaks ,page_positions, is_forward=True):

    if str(page) in page_breaks:
        return page_positions[page]
    if is_forward:
        bound = len(og_book_pdf)
        forward = page
        while (forward) <= bound:
            if str(forward) in page_breaks:
                return page_positions[forward]
            forward += 1
        return -1
    
    else:
        bound = 0
        backward = page
        while (backward) >= bound:
            if str(backward) in page_breaks:
                return page_positions[backward]
            backward -= 1
        return 0  # No valid page found


In [88]:
def add_indexes(latex_content, index): 
    matched = 0
    not_matched = 0
    not_found_terms = {}  # Dictionary to store terms not found along with page numbers

    for index_term, pages in tqdm(index.items()):
        # check = False
        # if index_term == 'application':
        #     check = True
        for page in pages:

            # upadte page number indexes as terms will be added

            page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', latex_content)
            page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', latex_content))}


            upper_bound = find_closest_page(page+1, page_breaks, page_positions ,True)
            lower_bound = find_closest_page(page-2, page_breaks, page_positions ,False)

            page_content = latex_content[lower_bound:upper_bound]
            
            # match = re.search(r'\b' + re.escape(index_term) + r'\b', page_content, re.IGNORECASE)
            match = re.search(  re.escape(index_term), page_content, re.IGNORECASE)

            # if check:
            #     print("Match found : " + match.group(0))
            #     print("Lowerbound : ", lower_bound)
            #     print("Upperbound : ", upper_bound)
            #     print("Page : ", page)
            if match:
                term_start = lower_bound + match.start()  # Adjust index relative to book_text
                term_end = term_start + len(index_term)
                indexed_term = "\index{" + index_term + "}"
                latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                matched += 1
            else:
                # try fuzzy matching
                term_length = len(index_term)
                max_dist = max(1, term_length // 4)  # 1/4rd of term length
                near_matches = find_near_matches(index_term, page_content, max_l_dist=max_dist)

                if near_matches:
                    best_match = min(near_matches, key=lambda x: x.dist)  # Get the closest match
                    term_start = lower_bound + best_match.start  # Adjust index relative to book_text
                    term_end = term_start + term_length
                    indexed_term = "\index{" + index_term + "}"
                    latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                    matched += 1
                    continue

                # did not find a match
                # put the index in the middle of the page
                index_position = lower_bound + (upper_bound - lower_bound) // 2
                indexed_term = "\index{" + index_term + "}"
                latex_content = latex_content[:index_position] + indexed_term + latex_content[index_position:]
                
                
                not_matched += 1
                if index_term not in not_found_terms:
                    not_found_terms[index_term] = []
                not_found_terms[index_term].append(page)


    print(f"Matched: {matched}, Not Matched: {not_matched}")
    return latex_content, not_found_terms

In [25]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v1 updating page_positions at every iteration

100%|██████████| 1749/1749 [00:14<00:00, 120.98it/s]

Matched: 1929, Not Matched: 347


In [89]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v1 updating page_positions at every iteration

100%|██████████| 1749/1749 [01:05<00:00, 26.55it/s]

Matched: 2273, Not Matched: 3


In [91]:
not_found_terms

{'Shiﬄett': [136], 'shuﬄing': [697], 'suﬃx arrays': [448]}

In [92]:
'suﬃx' == 'suffix'

False

## 3rd Book
## SLP

In [93]:
ROOT = "../../"

BOOK_PATH =  ROOT + "files/slp_book/slp-content.pdf"
TEX_PATH = ROOT + "files/slp_book/outputs/slp-content.tex"
INDEX_PATH = ROOT + "files/slp_book/slp-index.pdf"
OUTPUT_TEX_PATH = ROOT + "files/slp_book/outputs/slp-indexed-content.tex"

In [94]:
og_book_pdf = pymupdf.open(BOOK_PATH)
book_pdf = pymupdf.open(INDEX_PATH)

In [95]:
index_data = {}
for i in range(len(book_pdf)):
    page = book_pdf[i]
    index_data[i] = page.get_text("text")

In [96]:
pattern = re.compile(r"(.+?),\s*((?:\d+,?\s*)+)")

In [97]:
index = {}

for page, text in index_data.items():
    lines = [line.strip() for line in text.split("\n") if line.strip()]

    for line in lines:
        match = pattern.match(line)
        if match:
            term = match.group(1).strip()
            pages = [int(p) for p in re.findall(r"\d+", match.group(2))]
            index[term] = pages


In [98]:
with open(TEX_PATH, "r") as f:
    latex_content = f.read()

In [99]:
page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', latex_content)
page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', latex_content))}


In [100]:
def find_closest_page(page,page_breaks ,page_positions, is_forward=True):

    if str(page) in page_breaks:
        return page_positions[page]
    if is_forward:
        bound = len(og_book_pdf)
        forward = page
        while (forward) <= bound:
            if str(forward) in page_breaks:
                return page_positions[forward]
            forward += 1
        return -1
    
    else:
        bound = 0
        backward = page
        while (backward) >= bound:
            if str(backward) in page_breaks:
                return page_positions[backward]
            backward -= 1
        return 0  # No valid page found


In [101]:
def add_indexes(latex_content, index): 
    matched = 0
    not_matched = 0
    not_found_terms = {}  # Dictionary to store terms not found along with page numbers

    for index_term, pages in tqdm(index.items()):
        # check = False
        # if index_term == 'application':
        #     check = True
        for page in pages:

            # upadte page number indexes as terms will be added

            page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', latex_content)
            page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', latex_content))}


            upper_bound = find_closest_page(page+0, page_breaks, page_positions ,True)
            lower_bound = find_closest_page(page-1, page_breaks, page_positions ,False)

            page_content = latex_content[lower_bound:upper_bound]
            
            # match = re.search(r'\b' + re.escape(index_term) + r'\b', page_content, re.IGNORECASE)
            match = re.search(  re.escape(index_term), page_content, re.IGNORECASE)

            # if check:
            #     print("Match found : " + match.group(0))
            #     print("Lowerbound : ", lower_bound)
            #     print("Upperbound : ", upper_bound)
            #     print("Page : ", page)
            if match:
                term_start = lower_bound + match.start()  # Adjust index relative to book_text
                term_end = term_start + len(index_term)
                indexed_term = "\index{" + index_term + "}"
                latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                matched += 1
            else:
                # try fuzzy matching
                term_length = len(index_term)
                max_dist = max(1, term_length // 3)  # 1/3rd of term length
                near_matches = find_near_matches(index_term, page_content, max_l_dist=max_dist)

                if near_matches:
                    best_match = min(near_matches, key=lambda x: x.dist)  # Get the closest match
                    term_start = lower_bound + best_match.start  # Adjust index relative to book_text
                    term_end = term_start + term_length
                    indexed_term = "\index{" + index_term + "}"
                    latex_content = latex_content[:term_start] + indexed_term + latex_content[term_end:]
                    matched += 1
                    continue

                # did not find a match
                # put the index in the middle of the page
                index_position = lower_bound + (upper_bound - lower_bound) // 2
                indexed_term = "\index{" + index_term + "}"
                latex_content = latex_content[:index_position] + indexed_term + latex_content[index_position:]
                
                
                not_matched += 1
                if index_term not in not_found_terms:
                    not_found_terms[index_term] = []
                not_found_terms[index_term].append(page)


    print(f"Matched: {matched}, Not Matched: {not_matched}")
    return latex_content, not_found_terms

In [102]:
indexed_latex_content, not_found_terms= add_indexes(latex_content, index) #v1 updating page_positions at every iteration

100%|██████████| 1168/1168 [00:23<00:00, 49.93it/s] 

Matched: 1174, Not Matched: 84


In [103]:
not_found_terms

{'→(derives)': [389],
 '$ (RE end-of-line)': [8],
 'accessing a referent': [501],
 'ad hoc retrieval': [291],
 'adjacency pairs': [313],
 'afﬁx': [24],
 'agent, as thematic role': [462],
 'Viterbi': [372],
 'resolution of tag': [366],
 'Apple AIFF': [336],
 'conﬁdence': [285],
 '167': [207],
 'in smoothing': [48],
 'in IR': [291],
 'bias ampliﬁcation': [126],
 'blank in CTC': [342],
 'BM25': [291],
 'in IE': [441],
 'Brown corpus': [366],
 'original tagging of': [384],
 'history': [53],
 'Chomsky-adjunction': [395],
 'origin of term': [362],
 'collection in IR': [291],
 'commissive speech act': [312],
 'componential analysis': [476],
 'in relation extraction': [442],
 'constituency': [390],
 'titles which are not': [387],
 '392': [407],
 '186': [231],
 'gender agreement': [508],
 'recency preferences': [508],
 'treating low as zero': [379],
 'compared to HMM': [376],
 'Viterbi inference': [380],
 'decoding': [207],
 '(dev-test)': [39],
 'domination in syntax': [389],
 'minimum algorith

In [58]:
'afﬁx' == 'affix'

False

In [59]:
ord('ﬁ')


64257

In [ ]:
ord('ﬁ')


64257